# COVID-RELATED OSHA CITATION APPEALS

In [41]:
import pandas as pd
import os
import numpy as np

## Data import and vetting

I downloaded OSHRC's new listings report for July 2020 through December 2020 from https://www.oshrc.gov/listing-of-new-cases-received/ and imported the data. 

In [48]:
oshrcs_new_listings_2020 = pd.read_csv('oshrcs_new_listings_2020.csv')
oshrcs_new_listings_2020.head(3)

,Created Date,Docket Number,Case Name,Inspection City,Inspection State,OSHA Inspection Number
0,07/01/2020,20-0997,Premier Mobile Welding Inc,Fort Lauderdale,Florida,1466409
1,07/01/2020,20-0998,"JZ Construction, Inc.",Philadelphia,Pennsylvania,1456154
2,07/01/2020,20-0999,"Fenix Parts Inc, dba Leesville Auto",Rahway,New Jersey,1438846


I then copied data for covid-related inspections with citations from OSHA: https://www.osha.gov/enforcement/covid-19-data/inspections-covid-related-citations. I pasted the data in an Excel sheet and saved it as a csv. 

In [49]:
# Importing citation data
osha_covid_inspections = pd.read_csv('osha_covid_inspections.csv')

osha_covid_inspections.head(3)

,Establishment Name,City,State,Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited
0,BCP Ashland,ASHLAND,NEBRASKA,1494169,1/14/21,9446,1910.134(e)(1);1910.134(f)(2);1910.134(g)(1)(i...
1,MARY QUEEN & MOTHER CENTER,SAINT LOUIS,MISSOURI,1483017,1/12/21,13494,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2);1...
2,"Park View Rehab Center, LLC",CHICAGO,ILLINOIS,1483790,1/11/21,9446,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2)


Although the OSHA covid landing page (https://www.osha.gov/enforcement/covid-19-data/inspections-covid-related-citations) states that "All data is through April 18, 2021.", data for covid-related inspections with citations is available only through Jan 14, 2021. 

I converted the date column to datetime type. I sorted by the column to find the date of OSHA's earliest and latest covid-related citations. 

In [50]:
# Converting date col to datetime
osha_covid_inspections['Date Citation Issued'] = osha_covid_inspections['Date Citation Issued'].astype('datetime64[ns]')

print('Earliest covid-related inspection with citation')
display(osha_covid_inspections.sort_values('Date Citation Issued').head(1))
print('Latest covid-related inspection with citation')
display(osha_covid_inspections.sort_values('Date Citation Issued', ascending=False).head(1))

Earliest covid-related inspection with citation


,Establishment Name,City,State,Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited
309,"OHNH EMP, LLC",SALEM,OHIO,1475615,2020-07-13,13494,1910.134(c)(1);1910.134(e)(1)


Latest covid-related inspection with citation


,Establishment Name,City,State,Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited
0,BCP Ashland,ASHLAND,NEBRASKA,1494169,2021-01-14,9446,1910.134(e)(1);1910.134(f)(2);1910.134(g)(1)(i...


## Data Merge

In order to merge the two dataframes, I renamed the inspection id column and changed its data type. I also filtered the data to only include citations made between July and December 2020 (as the OSHRC data is only available for that time period). 

In [51]:
# Converting inspection id to str
osha_covid_inspections['Inspection Number'] = osha_covid_inspections['Inspection Number'].astype('str')

# Renaming inspection id col
osha_covid_inspections = osha_covid_inspections.rename(columns={'Inspection Number': 'OSHA Inspection Number'})

# Filtering by date
osha_covid_inspections = osha_covid_inspections[osha_covid_inspections['Date Citation Issued'] < '2021-01-01']

osha_covid_inspections.head(3)

,Establishment Name,City,State,OSHA Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited
9,"THE LUTHERAN HOME, INC.",WAUWATOSA,WISCONSIN,1484381,2020-12-28,13494,1910.134(e)(1)
10,Val Verde Regional Medical Center,DEL RIO,TEXAS,1485015,2020-12-28,11567,1904.39(a)(1);1904.4(a)
11,Geodis,COLUMBUS,OHIO,1483526,2020-12-23,13494,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2)


I then performed a left merge of the two dataframes on the inspection id column. 

In [52]:
# Merging dfs
merged = pd.merge(osha_covid_inspections, oshrcs_new_listings_2020, on='OSHA Inspection Number', how='left')

merged[merged['Docket Number'].isnull() == False].head(3)

,Establishment Name,City,State,OSHA Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited,Created Date,Docket Number,Case Name,Inspection City,Inspection State
43,"2509 South Fourth Operating, LLC",PHILADELPHIA,PENNSYLVANIA,1490507,2020-12-01,12145,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2),12/28/2020,20-1681,"2509 South Fourth Operating, LLC, dba St. Moni...",Philadelphia,Pennsylvania
45,"FRIENDSHIP SENIOR OPTIONS, NFP",SCHAUMBURG,ILLINOIS,1478469,2020-11-25,13494,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2),12/18/2020,20-1670,"Friendship Senior Options, NFP",Schaumburg,Illinois
46,"Providence Rest, Inc.",BRONX,NEW YORK,1488827,2020-11-25,13494,1910.134(e)(1);1910.134(f)(2),12/21/2020,20-1673,"Providence Rest, Inc.",Bronx,New York


## Data Analysis

I indetified OSHA citations which were brought before the OSHRC by creating a new 'appeal_status' column.

In [53]:
merged['appeal_status'] = np.where(merged['Docket Number'].isnull() == False, 'appealed', 'not appealed')

merged.head(3)

,Establishment Name,City,State,OSHA Inspection Number,Date Citation Issued,Total Initial Penalty,Standards Cited,Created Date,Docket Number,Case Name,Inspection City,Inspection State,appeal_status
0,"THE LUTHERAN HOME, INC.",WAUWATOSA,WISCONSIN,1484381,2020-12-28,13494,1910.134(e)(1),NaN,NaN,NaN,NaN,NaN,not appealed
1,Val Verde Regional Medical Center,DEL RIO,TEXAS,1485015,2020-12-28,11567,1904.39(a)(1);1904.4(a),NaN,NaN,NaN,NaN,NaN,not appealed
2,Geodis,COLUMBUS,OHIO,1483526,2020-12-23,13494,1910.134(c)(1);1910.134(e)(1);1910.134(f)(2),NaN,NaN,NaN,NaN,NaN,not appealed


Finally, I created a separate table with summary counts for appeals. 

In [54]:
appeal_counts = merged.appeal_status.value_counts().reset_index()

appeal_counts['pct_of_total'] = appeal_counts.appeal_status / appeal_counts.appeal_status.sum() * 100

appeal_counts

,index,appeal_status,pct_of_total
0,not appealed,164,54.48505
1,appealed,137,45.51495


# Findings
The data indicates that about 45 percent of employers cited by OSHA in 2020 for covid-related violations appealed to OSHRC. 